<h1><center><strong>Tarefas</strong></center></h1>

Notebook respondendo as tarefas propostas para a atividade:

**1.** Plotar gráficos dos preços e retornos e tecer comentários sobre possível
heterocedasticidade condicional;

**2.** Ajustar modelos de volatilidade univariados e escolher o mais adequado (pode usar
critérios de informação e/ou validação cruzada);


**3.** Prever a volatilidade um passo à frente usando o modelo selecionado no item anterior;


**4.** Comparar volatilidades entre os retornos selecionados (quais são maiores e menores,
relacionando com algum storytelling);


**5.** Calcular retorno do portfólio (usando pesos designados inicialmente) e ajustar um modelo
de volatilidade para o mesmo, realizando a previsão um passo à frente da volatilidade do
portfólio;


**6.** Calcular betas individuais usando regressão simples (estático) e tecer comentários;


**7.** Calcular beta estático de um portfolio contendo os ativos selecionados e pesos
determinados por vocês, assumindo as hipóteses do CAPM;


**8.** Fazer uma otimização dos pesos do seu portfólio inicial e repetir as etapas 5 e 7,
comparando o seu portfólio inicial com o otimizado. A otimização será abordada no lab.


### **Bibliotecas**

In [1]:
# Importando Bibliotecas
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

### **Gráficos e Funções**

Para manter o notebook mais claro e legível, todos os gráficos e funções serão criados nesta seção.

In [2]:
def plot_interactive_stock_chart(data, title='Preço de Fechamento das Ações'):
    """
    Plota um gráfico interativo com um menu dropdown para selecionar ações.

    Parâmetros:
    - data (dict): Um dicionário onde as chaves são os nomes das ações e os valores são os DataFrames com as colunas 'Close'.
    - title (str): O título do gráfico.
    """
    # Criando o gráfico
    fig = go.Figure()

    # Cores específicas para cada ação
    colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'orange']

    # Adicionando as séries de dados ao gráfico com cores específicas
    for (key, value), color in zip(data.items(), colors):
        fig.add_trace(
            go.Scatter(x=value.index, y=value, visible=False, name=key, line=dict(color=color))
        )

    # Deixando a primeira série visível
    fig.data[0].visible = True

    # Criando os botões para o menu dropdown
    buttons = []
    for i, key in enumerate(data.keys()):
        buttons.append(
            dict(
                label=key,
                method='update',
                args=[{'visible': [False] * len(data)},
                      {'title': f'Preço de fechamento: {key}'}],
                )
        )
        buttons[-1]['args'][0]['visible'][i] = True  # Toggle i-th trace to "visible"

    # Adicionando o menu dropdown ao gráfico
    fig.update_layout(
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top'
        }],
        title=title,
        template='plotly_dark'
    )

    fig.show()

In [3]:
def plot_stocks(data, title='Preço de fechamento das ações'):
    """
    Plota um gráfico com os preços de fechamento para múltiplas ações.
    
    Parâmetros:
    - data (dict): Um dicionário onde as chaves são os nomes das ações e os valores são os DataFrames com as colunas 'Close'.
    - title (str): O título do gráfico.
    """
    # Criando o gráfico
    fig = go.Figure()

    # Cores específicas para cada ação
    colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'orange']

    # Adicionando as séries de dados ao gráfico, cada uma com uma cor específica
    for (key, value), color in zip(data.items(), colors):
        fig.add_trace(
            go.Scatter(x=value.index, y=value, name=key, line=dict(color=color))
        )

    # Configurando o layout do gráfico
    fig.update_layout(
        title=title,
        xaxis_title='Data',
        yaxis_title='Preço de Fechamento',
        template='plotly_dark',
        legend_title='Ações'
    )

    fig.show()

In [14]:
def plot_log_returns(dataframes):
    """
    Plota um gráfico interativo com retornos logarítmicos para várias ações.
    O usuário pode selecionar a série de dados desejada através de um dropdown.
    
    Args:
    dataframes (dict): Dicionário de pandas DataFrames com as séries temporais de preços.
    """
    # Lista de cores para diferenciar visualmente cada série de dados
    cores = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'orange']

    # Criando o gráfico
    fig = go.Figure()

    # Adicionando uma trace para cada DataFrame, com cores específicas
    for i, (nome, df) in enumerate(dataframes.items()):
        df.reset_index(inplace=True)  # Resetando o índice para usar a data como uma coluna
        fig.add_trace(
            go.Scatter(x=df['Date'], y=df['Log_Return'],
                       name=f'Retorno Logarítmico {nome}',
                       visible=False,  # Inicia invisível
                       line=dict(color=cores[i % len(cores)])  # Cor específica para cada ação
            )
        )

    # Definindo a primeira trace para ser visível inicialmente
    fig.data[0].visible = True

    # Criando botões de dropdown para a escolha da ação
    buttons = []

    # Botão para cada ação que atualiza a visibilidade das traces
    for i, nome in enumerate(dataframes.keys()):
        button = dict(
            label=nome,
            method="update",
            args=[{"visible": [j == i for j in range(len(dataframes))]},
                  {"title": f"Retornos Logarítmicos para {nome}"}]
        )
        buttons.append(button)

    # Adicionando os botões ao layout
    fig.update_layout(
        updatemenus=[dict(
            active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )],
        template="plotly_dark",
        title_text=f"Retornos Logarítmicos para {list(dataframes.keys())[0]}",
        xaxis_title="Data",
        yaxis_title="Retorno Logarítmico",
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        font_color="white"
    )

    # Configurações adicionais para melhor visualização no tema escuro
    fig.update_xaxes(showline=True, linewidth=1, linecolor='white', gridcolor='grey')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='white', gridcolor='grey')

    fig.show()



### **Dados**

Importando histórico de dados das empresas analisadas.

In [4]:
# Importando Base de Dados em um dicionario
df_AURA33 = pd.read_csv("data\\df_AURA33.csv", parse_dates=True, index_col="Date")
df_AURA33.index = pd.to_datetime(df_AURA33.index, utc=True)

df_CBAV3 = pd.read_csv("data\\df_CBAV3.csv", parse_dates=True, index_col="Date")
df_CBAV3.index = pd.to_datetime(df_CBAV3.index, utc=True)

df_CMIN3 = pd.read_csv("data\\df_CMIN3.csv", parse_dates=True, index_col="Date")
df_CMIN3.index = pd.to_datetime(df_CMIN3.index, utc=True)

df_CSNA3 = pd.read_csv("data\\df_CSNA3.csv", parse_dates=True, index_col="Date")
df_CSNA3.index = pd.to_datetime(df_CSNA3.index, utc=True)

df_GGBR4 = pd.read_csv("data\\df_GGBR4.csv", parse_dates=True, index_col="Date")
df_GGBR4.index = pd.to_datetime(df_GGBR4.index, utc=True)

df_USIM5 = pd.read_csv("data\\df_USIM5.csv", parse_dates=True, index_col="Date")
df_USIM5.index = pd.to_datetime(df_USIM5.index, utc=True)

df_VALE3 = pd.read_csv("data\\df_VALE3.csv", parse_dates=True, index_col="Date")
df_VALE3.index = pd.to_datetime(df_VALE3.index, utc=True)

In [5]:
# Lista dos nomes dos arquivos CSV
arquivos = ['df_AURA33', 'df_CBAV3', 'df_CMIN3', 'df_CSNA3', 'df_GGBR4', 'df_USIM5', 'df_VALE3']

# Criando um dicionário onde a chave é o nome do arquivo e o valor é o DataFrame correspondente
dataframes = {nome: pd.read_csv(f"data\\{nome}.csv", parse_dates=True, index_col="Date") for nome in arquivos}

# Convertendo os índices de cada DataFrame para datetime com timezone UTC
for df in dataframes.values():
    df.index = pd.to_datetime(df.index, utc=True)

In [6]:
dataframes["df_AURA33"]

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-06 03:00:00+00:00,44.011464,44.011464,39.961329,41.570583,508500,0.0,0.0
2020-07-07 03:00:00+00:00,41.570587,42.120324,40.903664,41.850857,57000,0.0,0.0
2020-07-08 03:00:00+00:00,41.850853,45.901525,41.850853,44.281471,288000,0.0,0.0
2020-07-09 03:00:00+00:00,44.281475,47.510783,43.201440,44.280937,141000,0.0,0.0
2020-07-10 03:00:00+00:00,44.280934,44.281471,43.201436,44.281471,157500,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-10 03:00:00+00:00,45.599998,47.500000,45.150002,47.490002,96355,0.0,0.0
2024-06-11 03:00:00+00:00,46.099998,47.770000,46.099998,47.709999,46775,0.0,0.0
2024-06-12 03:00:00+00:00,47.790001,48.189999,47.029999,47.750000,91256,0.0,0.0


In [7]:
for key, value in dataframes.items():
    print(f"{key}: {value.index.min()}")

df_AURA33: 2020-07-06 03:00:00+00:00
df_CBAV3: 2021-07-16 03:00:00+00:00
df_CMIN3: 2021-02-22 03:00:00+00:00
df_CSNA3: 2000-01-03 02:00:00+00:00
df_GGBR4: 2000-01-03 02:00:00+00:00
df_USIM5: 2000-01-03 02:00:00+00:00
df_VALE3: 2000-01-03 02:00:00+00:00


A menor data disponível dentre as ações selecionadas para o trabalho é 16/07/2021. Portanto, iremos filtrar todos os dataframes para trabalhar com dados a partir de 08/2021, facilitando assim a comparação entre as ações.

In [8]:
# Filtrando os DataFrames para datas a partir de "2021-08-01"
for nome, df in dataframes.items():
    dataframes[nome] = df.loc["2021-08-01":]

### **Tarefas**

#### **1.** Plotar gráficos dos preços e retornos e tecer comentários sobre possível heterocedasticidade condicional;

Primeiramente plotando os gráficos de preços:

In [9]:
data = {
    'AURA33': dataframes["df_AURA33"]['Close'],
    'CBAV3': dataframes["df_CBAV3"]['Close'],
    'CMIN3': dataframes["df_CMIN3"]['Close'],
    'CSNA3': dataframes["df_CSNA3"]['Close'],
    'GGBR4': dataframes["df_GGBR4"]['Close'],
    'USIM5': dataframes["df_USIM5"]['Close'],
    'VALE3': dataframes["df_VALE3"]['Close']
}

plot_stocks(data, 'Preço de Fechamento das Ações Selecionadas')

Como temos ações com diferentes faixas de preços, os preços também podem ser visualizados de acordo com cada tick no gráfico abaixo:

In [10]:
# Criando dataframe de plotagem
data = {
    'AURA33': dataframes["df_AURA33"]['Close'],
    'CBAV3': dataframes["df_CBAV3"]['Close'],
    'CMIN3': dataframes["df_CMIN3"]['Close'],
    'CSNA3': dataframes["df_CSNA3"]['Close'],
    'GGBR4': dataframes["df_GGBR4"]['Close'],
    'USIM5': dataframes["df_USIM5"]['Close'],
    'VALE3': dataframes["df_VALE3"]['Close']
}

# Aplicando função que cria o gráfico
plot_interactive_stock_chart(data, 'Preço de Fechamento das Ações Selecionadas')

O próximo passo para esta primeira questão é calcular os logs dos retornos para cada uma das ações e plotar o gráfico para verificar o comportamento.

In [11]:
# Calculando os retornos logarítmicos para cada DataFrame no dicionário
for nome, df in dataframes.items():
    df["log_close"] = np.log(df["Close"])
    df["Log_Return"] = df["log_close"].diff().dropna()  
    df.drop(columns=["log_close"], inplace=True)  


C:\Users\GBinfo\AppData\Local\Temp\ipykernel_6936\4233973933.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\GBinfo\AppData\Local\Temp\ipykernel_6936\4233973933.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\GBinfo\AppData\Local\Temp\ipykernel_6936\4233973933.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [15]:
# Plotando gráfico dos retornos
plot_log_returns(dataframes)


Observando os gráficos dos retornos, percebe-se que eles parecem não possuir uma tendência e se assemelham a um ruído branco. Isso indica que as séries analisadas apresentam períodos com diferentes volatilidades, sugerindo a presença de heterocedasticidade condicional.